In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import sys
import os
sys.path.append(os.path.abspath('/content/drive/MyDrive/02 Research/01 Affective Computing/03 Benchmark/'))
try:
    import neurokit2
except ImportError as e:
    !pip install neurokit2

try:
    import pyedflib
except ImportError as e:
    !pip install pyedflib

try:
    import tpot
except ImportError as e:
    !pip install tpot


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 KB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11954 sha256=1818d035b9dd1d13846cde27bf3861e876a1c3a62bda6bf264928696cda95c9d
  Stored in directory: /root/.cache/pip/wheels/48/8c/93/3afb1916772591fe6bcc25cdf8b1c5bdc362f0ec8e2f0fd413
Successfully built stopit


# Hyperparameters

In [3]:
num_classes = 2
modality_included = None
window_length = 5
task = 'Arousal'

# Dataset

In [4]:
from pyAffeCT.datasets import WESAD, BioVidEmo, BioVid, BIRAFFE2

# constructing a dataset instance
#dataset = WESAD('/content/drive/MyDrive/02 Research/01 Affective Computing/03 Benchmark/pyAffeCT/data/02 WESAD/WESAD')
#dataset = BioVidEmo('/content/drive/MyDrive/02 Research/01 Affective Computing/03 Benchmark/pyAffeCT/data/04 BioVidEmo/bio_raw')
#dataset = BioVid('/content/drive/MyDrive/02 Research/01 Affective Computing/03 Benchmark/pyAffeCT/data/03 BioVid')
dataset = BIRAFFE2('/content/drive/MyDrive/02 Research/01 Affective Computing/03 Benchmark/pyAffeCT/data/01 BIRAFFE2')
#dataset = ASCERTAIN()

# view the subject id list
subject_list = dataset.subject_list



# Traditional Machine Learning

In [5]:
# # -*- coding: utf-8 -*-
# """
# Created on Fri Mar  3 19:05:30 2023

# @author: rjfan
# """

from pyAffeCT.models import traditionalClassifier
from pyAffeCT.metrics import classificationEvaluator


# prepare for the dataframe, set number of classess to be 3
df_train, df_test = dataset.prepare_dataframe(subject_included=subject_list[:], split_by_subject=True, modality_included=modality_included, window_length=window_length, label_task=task, num_classes=num_classes)
df_train.head()

X_train = df_train.loc[:,df_train.columns!='Label']
y_train = df_train['Label']
X_test = df_test.loc[:,df_test.columns!='Label']
y_test = df_test['Label']

# instantiate tranditional classifier
clf = traditionalClassifier()

# fit to the training set, tuning to the best hyperparameters
clf.tune_hyperparams(X_train,y_train)

# predict to classess and probabilities
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)

# instantiate evaluator and evaluate results
evaluator = classificationEvaluator()
results = evaluator.evaluate(y_test, y_prob=y_prob, plot_roc=False)
print(results)


100%|██████████| 102/102 [39:02<00:00, 22.97s/it]


                SVM       GNB       BNB       MNB        DT        RF  \
accuracy   0.491732  0.503174  0.502255  0.503257  0.510189  0.494488   
precision  0.492154  0.502242  0.501572  0.502633  0.510442  0.494505   
recall     0.518624  0.710874  0.719392  0.621847  0.498079  0.496075   
f1         0.505042  0.588618  0.591053  0.555921  0.504185  0.495289   
roc_auc    0.497296  0.508118  0.504100  0.501355  0.508953  0.495739   
pr_auc     0.499785  0.509591  0.502725  0.499887  0.505754  0.503043   

                KNN  Extra Tree   Bagging  AdaBoost   XGBoost  
accuracy   0.511274    0.498246  0.494655  0.502088  0.498413  
precision  0.511060    0.498287  0.493130  0.501910  0.498625  
recall     0.520962    0.510272  0.383665  0.548522  0.575580  
f1         0.515964    0.504209  0.431564  0.524182  0.534346  
roc_auc    0.511274    0.503401  0.497103  0.505279  0.498021  
pr_auc     0.505762    0.507600  0.497078  0.506182  0.504173  


# Deep Learning

## Dataset reading

In [6]:

# prepare for the dataframe, set number of classess to be 3
df_train, df_test = dataset.prepare_flatten_dataframe(subject_included=subject_list[:], split_by_subject=True, modality_included = modality_included, window_length=window_length,label_task=task, num_classes=num_classes)

train_loader, test_loader, input_shape = dataset.df_to_dataloader(df_train,df_test,num_classes, batch_size = 4)


 27%|██▋       | 28/102 [04:51<12:49, 10.39s/it]


TypeError: ignored

# Model

## MLP

In [ ]:
from pyAffeCT.models import MLP

net = MLP(input_shape = input_shape, num_classes=num_classes)
net.init_weights(method=None)
_,_ = net.train(train_loader, test_loader)

## FCN

In [ ]:
from pyAffeCT.models import FCN

net = FCN(input_shape = input_shape, num_classes=num_classes)
net.init_weights(method=None)
_,_ = net.train(train_loader, test_loader)

## ResNet

In [ ]:
from pyAffeCT.models import ResNet

net = ResNet(input_shape = input_shape, num_classes=num_classes)
net.init_weights(method=None)
_,_ = net.train(train_loader, test_loader)

## RNN

In [ ]:
from pyAffeCT.models import RNN

net = RNN(input_shape = input_shape, num_classes=num_classes)
net.init_weights(method=None)
_,_ = net.train(train_loader, test_loader)

## LSTM

In [ ]:
from pyAffeCT.models import LSTM

net = LSTM(input_shape = input_shape, num_classes=num_classes)
net.init_weights(method=None)
_,_ = net.train(train_loader, test_loader)

## GRU

In [ ]:
from pyAffeCT.models import GRU

net = GRU(input_shape = input_shape, num_classes=num_classes)
net.num_epochs = 30
net.init_weights(method=None)
_,_ = net.train(train_loader, test_loader)

## RNN_FCN

In [ ]:
from pyAffeCT.models import RNN_FCN

net = RNN_FCN(input_shape = input_shape, num_classes=num_classes)
net.init_weights(method=None)
_,_ = net.train(train_loader, test_loader)

## LSTM_FCN

In [ ]:
from pyAffeCT.models import LSTM_FCN

net = LSTM_FCN(input_shape = input_shape, num_classes=num_classes)
net.init_weights(method=None)
_,_ = net.train(train_loader, test_loader)

## GRU_FCN

In [ ]:
from pyAffeCT.models import GRU_FCN

net = GRU_FCN(input_shape = input_shape, num_classes=num_classes)
net.init_weights(method=None)
_,_ = net.train(train_loader, test_loader)